<a href="https://colab.research.google.com/github/BrunoSSM/Alura-IAGemini-buscaremprego/blob/main/alerta_oportunidades_profissionais1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Projeto: Alerta de Oportunidades Profissionais com Gemini e Agentes
Este projeto usa agentes Gemini para buscar notícias sobre oportunidades profissionais, filtrar as mais relevantes, gerar alertas e sugerir ações para o usuário.



In [1]:
# Instalar dependências
%pip install -q google-genai google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [2]:
# Configuração da API do Gemini
import os
from google.colab import userdata
from google import genai

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"


In [3]:
# Importações principais
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
from IPython.display import Markdown, display
import textwrap

def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text:
                    final_response += part.text + "\n"
    return final_response

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


In [4]:
# Agente: Busca oportunidades
def agente_buscador_oportunidades(area, data_de_hoje):
    buscador = Agent(
        name="agente_oportunidades",
        model="gemini-2.0-flash",
        instruction="""
Você é um agente que busca notícias sobre oportunidades de trabalho, bolsas de estudo, concursos públicos e programas de aceleração relacionados à área abaixo.
Busque as informações mais recentes, com no máximo 30 dias.
Selecione as 3 a 5 mais relevantes.
""",
        description="Busca oportunidades profissionais recentes",
        tools=[google_search]
    )
    entrada = f"Área: {area}\nData: {data_de_hoje}"
    return call_agent(buscador, entrada)


In [5]:
# Agente: Gera plano de ação
def agente_estrategista(area, oportunidades_texto):
    estrategista = Agent(
        name="agente_estrategista",
        model="gemini-2.0-flash",
        instruction="""
Você é um estrategista de carreira.
Com base nas oportunidades listadas abaixo, elabore um pequeno plano de ação para que o usuário possa se preparar ou se inscrever em cada uma delas.
""",
        description="Gera estratégias baseadas em oportunidades"
    )
    entrada = f"Área: {area}\nOportunidades:\n{oportunidades_texto}"
    return call_agent(estrategista, entrada)


In [6]:
# Executa o sistema
area = input("Digite sua área de interesse profissional (ex: tecnologia, saúde, educação): ")
data_hoje = date.today().strftime("%d/%m/%Y")

if not area:
    print("⚠️ Nenhuma área informada.")
else:
    print("🔍 Buscando oportunidades...")
    oportunidades = agente_buscador_oportunidades(area, data_hoje)
    display(Markdown("### 📰 Oportunidades encontradas:"))
    display(to_markdown(oportunidades))

    print("🧠 Gerando plano de ação...")
    plano = agente_estrategista(area, oportunidades)
    display(Markdown("### 📌 Plano de Ação Recomendado:"))
    display(to_markdown(plano))


Digite sua área de interesse profissional (ex: tecnologia, saúde, educação): Segurança da informação 
🔍 Buscando oportunidades...


### 📰 Oportunidades encontradas:

> Para encontrar oportunidades recentes na área de Segurança da Informação, vou realizar algumas pesquisas direcionadas.
> 
> 
> Com base nas pesquisas realizadas, encontrei as seguintes oportunidades e informações relevantes na área de Segurança da Informação:
> 
> 1.  **Bolsa de Estudos - Pós-Graduação em Segurança da Informação - CECyber:** A CECyber está oferecendo bolsas de estudos totais e parciais para a pós-graduação em Segurança da Informação e Inteligência Defensiva com início em Junho de 2025. As inscrições estão abertas de 01/04/2025 a 09/05/2025. ([1])
> 
> 2.  **Concurso Público 1/2025 - Tribunal de Justiça Militar:** Este concurso oferece 2 vagas para analista de segurança da informação judiciário, além de outras vagas na área de TI. ([2])
> 
> 3.  **Vaga para Analista de Tecnologia da Informação - Segurança da Informação - PPSA - 2025:** Há uma vaga para Analista de Tecnologia da Informação com foco em Segurança da Informação na PPSA (Pré-Sal Petróleo) com edital nº 1 para 2025. ([7])
> 
> 4.  **Concursos TI - Editais Publicados e Iminentes:** O Blog Gran Cursos Online divulgou em 08/04/2025 informações sobre concursos na área de TI com editais já publicados e outros previstos. O Tribunal de Justiça do Estado do Rio de Janeiro (TJ RJ) deve ofertar vagas para Analista de Segurança da Informação. Além disso, o Banco do Estado do Rio Grande do Sul (Banrisul) publicou edital para profissionais da área de Tecnologia da Informação, com 100 vagas de nível superior. ([8])
> 
> 5.  **Oportunidades no Portal do Servidor - GOV.BR:** O Portal do Servidor oferece vagas de nível superior na área de Privacidade e Segurança da Informação para servidores públicos estáveis. ([4])
> 
> Além disso, vale a pena conferir:
> 
> *   **Concursos 2025 (Blog Gran Cursos Online):** Levantamento com mais de 600 editais previstos em diversas carreiras, incluindo 2.732 vagas para Tecnologia da Informação. ([12])
> *   **Programa de Aceleração Espaço Finep:** Programa de aceleração para startups, com espaço físico disponibilizado pela Finep no Rio de Janeiro. As startups aceleradas obrigam-se a obedecer, cumprir e fazer com que os seus colaboradores observem e cumpram, com absoluto rigor, o Código de Ética, Conduta e Integridade da Finep, as normas internas desta empresa pública, em especial aquelas relacionadas à ética, integridade e segurança da informação. ([17])
> *   **Analista de Privacidade e Analista de Cibersegurança em alta (Forbes):** O LinkedIn divulgou as 25 posições que tiveram maior crescimento de demanda nos últimos cinco anos, incluindo Analista de Privacidade e Analista de Cibersegurança. ([10])
> 
> É importante verificar os requisitos de cada oportunidade e os prazos de inscrição para garantir a participação nos processos seletivos.
> 


🧠 Gerando plano de ação...


### 📌 Plano de Ação Recomendado:

> ## Plano de Ação Estratégico para Oportunidades em Segurança da Informação (2025)
> 
> Com base nas oportunidades identificadas, sugiro o seguinte plano de ação para te preparar e maximizar suas chances de sucesso:
> 
> **1. Bolsa de Estudos - Pós-Graduação em Segurança da Informação - CECyber:**
> 
> *   **Ação Imediata:** Analisar detalhadamente o edital da bolsa da CECyber. Verificar os critérios de elegibilidade, documentação necessária e os temas abordados na pós-graduação.
> *   **Ação Contínua:**  Começar a reunir toda a documentação necessária. Se houver alguma lacuna em seu currículo que possa ser um diferencial (ex: certificações, projetos, experiência), investir em melhorar esses pontos.
> *   **Ação de Preparação:** Se a bolsa exigir alguma prova ou entrevista, buscar materiais de estudo e praticar entrevistas.
> *   **Prazo Final:** Inscrever-se até 09/05/2025.
> 
> **2. Concurso Público 1/2025 - Tribunal de Justiça Militar:**
> 
> *   **Ação Imediata:** Analisar o edital do concurso do Tribunal de Justiça Militar. Identificar o conteúdo programático específico para o cargo de Analista de Segurança da Informação Judiciário.
> *   **Ação Contínua:** Montar um plano de estudos focado no conteúdo do edital. Priorizar as áreas de maior peso na prova. Utilizar materiais de estudo como livros, cursos online e questões de concursos anteriores.
> *   **Ação de Preparação:** Resolver questões de concursos anteriores da banca organizadora (se conhecida). Fazer simulados para testar seus conhecimentos e identificar áreas que precisam de mais estudo.
> *   **Ação de Preparação:** Se preparar para o TAF (Teste de Aptidão Física) caso haja no concurso.
> *   **Prazo:** Acompanhar as datas de inscrição e prova do concurso.
> 
> **3. Vaga para Analista de Tecnologia da Informação - Segurança da Informação - PPSA - 2025:**
> 
> *   **Ação Imediata:** Analisar o edital nº 1/2025 da PPSA para o cargo de Analista de TI com foco em Segurança da Informação. Verificar os requisitos, atribuições e o processo seletivo.
> *   **Ação Contínua:** Atualizar seu currículo com as habilidades e experiências relevantes para a vaga. Preparar uma carta de apresentação destacando suas qualificações e o seu interesse pela PPSA.
> *   **Ação de Preparação:** Pesquisar sobre a PPSA e o setor de petróleo e gás. Estudar os temas técnicos relacionados à segurança da informação em ambientes de infraestrutura crítica.
> *   **Prazo:** Acompanhar as datas de inscrição e processo seletivo da PPSA.
> 
> **4. Concursos TI - Editais Publicados e Iminentes (TJ RJ e Banrisul):**
> 
> *   **Ação Imediata:** Acessar os editais do TJ RJ (quando publicado) e do Banrisul. Analisar os requisitos, conteúdo programático e as etapas dos concursos.
> *   **Ação Contínua:** Seguir as ações de preparação para concursos descritas no item 2, adaptando-as para os conteúdos específicos de cada concurso.
> *   **Prazo:** Acompanhar as datas de inscrição e prova de cada concurso.
> 
> **5. Oportunidades no Portal do Servidor - GOV.BR:**
> 
> *   **Ação Imediata:** Acessar o Portal do Servidor e verificar as vagas disponíveis na área de Privacidade e Segurança da Informação.
> *   **Ação Contínua:** Manter seu currículo atualizado no Portal do Servidor.
> *   **Prazo:** Monitorar o Portal do Servidor regularmente.
> 
> **Ações Complementares:**
> 
> *   **Concursos 2025 (Blog Gran Cursos Online):** Consultar o levantamento do Gran Cursos Online para identificar outros concursos na área de TI que possam ser do seu interesse.
> *   **Programa de Aceleração Espaço Finep:** Se você tem uma startup na área de segurança da informação, considerar participar do programa de aceleração da Finep.
> *   **Analista de Privacidade e Analista de Cibersegurança em alta (Forbes):** Continuar se atualizando sobre as tendências do mercado de trabalho na área de segurança da informação, com foco nas áreas de privacidade e cibersegurança.
> 
> **Dicas Gerais:**
> 
> *   **Networking:** Participe de eventos, grupos de discussão e fóruns online sobre segurança da informação.
> *   **Certificações:** Considere obter certificações relevantes na área de segurança da informação.
> *   **Projetos Pessoais:** Desenvolva projetos pessoais na área de segurança da informação para demonstrar suas habilidades e conhecimentos.
> 
> Lembre-se que este é um plano de ação inicial. É importante adaptá-lo às suas necessidades e objetivos específicos. Boa sorte!
> 
